Récupération d'un csv champ ( stat + description + url image)

In [1]:
from requests import get
import requests
from bs4 import BeautifulSoup
import re
import time
import polars as pl
from sklearn.model_selection import train_test_split
import seaborn as sns
import joblib
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier

On récupère le csv classique

In [2]:
import requests
import polars as pl

def get_lol_champions():
    # 1. Récupération du dernier numéro de patch
    try:
        version_url = "https://ddragon.leagueoflegends.com/api/versions.json"
        latest_version = requests.get(version_url).json()[0]
        print(f"Chargement des données pour le patch : {latest_version}")
    except Exception as e:
        print(f"Erreur version : {e}")
        return None

    # 2. Télécharger les données
    url = f"https://ddragon.leagueoflegends.com/cdn/{latest_version}/data/en_US/champion.json"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erreur API (Status: {response.status_code})")
        return None

    data = response.json()
    champions_dict = data['data']

    # 3. On chope les infos
    rows = []
    for champ_id, details in champions_dict.items():
        stats = details['stats']
        info = details['info']
        image_file = details['image']['full'] 
        
        rows.append({
            "Name": details['name'], 
            "InternalName": champ_id, # Le nom interne pour la jointure
            "Title": details['title'], 
            "Class": details['tags'][0],
            
            # Liens Images
            "ImageURL": f"https://ddragon.leagueoflegends.com/cdn/{latest_version}/img/champion/{image_file}",
            
            # Stats
            "AttackRange": stats['attackrange'],
            "MoveSpeed": stats['movespeed'],
            "HP": stats['hp'],
            "HPperLevel": stats['hpperlevel'],
            "hpregen": stats['hpregen'],
            "hpregenperlevel": stats['hpregenperlevel'],
            "Armor": stats['armor'],
            "ArmorperLevel": stats['armorperlevel'],
            "AttackDamage": stats['attackdamage'],
            "AttackDamageperLevel": stats['attackdamageperlevel'],
            "attackspeed": stats['attackspeed'],
            "attackspeedperlevel": stats['attackspeedperlevel'],
            "Spellblock": stats['spellblock'],
            "spellblockperlevel": stats['spellblockperlevel'],
            "Prv_attack": info['attack'],
            "Prv_defense": info['defense'],
            "Prv_magic": info['magic']
        })

    # 4. Créer le Tableau Polars
    tableau_champ = pl.DataFrame(rows)
    # Récupération réussie
    return tableau_champ 

if __name__ == "__main__":
    df_final = get_lol_champions()

    if df_final is not None:
        print("Aperçu du tableau :")
        print(df_final.head(5))
        

   

Chargement des données pour le patch : 16.2.1
Aperçu du tableau :
shape: (5, 22)
┌─────────┬────────────┬────────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Name    ┆ InternalNa ┆ Title      ┆ Class    ┆ … ┆ spellbloc ┆ Prv_attac ┆ Prv_defen ┆ Prv_magic │
│ ---     ┆ me         ┆ ---        ┆ ---      ┆   ┆ kperlevel ┆ k         ┆ se        ┆ ---       │
│ str     ┆ ---        ┆ str        ┆ str      ┆   ┆ ---       ┆ ---       ┆ ---       ┆ i64       │
│         ┆ str        ┆            ┆          ┆   ┆ f64       ┆ i64       ┆ i64       ┆           │
╞═════════╪════════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Aatrox  ┆ Aatrox     ┆ the Darkin ┆ Fighter  ┆ … ┆ 2.05      ┆ 8         ┆ 4         ┆ 3         │
│         ┆            ┆ Blade      ┆          ┆   ┆           ┆           ┆           ┆           │
│ Ahri    ┆ Ahri       ┆ the Nine-T ┆ Mage     ┆ … ┆ 1.3       ┆ 3         ┆ 4         ┆ 8         │
│         

In [7]:
#récupération ds listes de vérif et utiles
liste_noms = df_final['InternalName'].to_list()
liste_urls = df_final['ImageURL'].to_list()
#sauvegarde du dataframe complet
df_final.write_csv("présentation_champions_stat.csv", separator=";")


second data frame détaillé

In [8]:
import requests
import polars as pl

# 1. Version Auto
def get_latest_version():
    try:
        return requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    except:
        return "16.1.1"

VERSION = get_latest_version()
print(f"Patch : {VERSION}")

# URLs de base
BASE_URL = f"https://ddragon.leagueoflegends.com/cdn/{VERSION}/data/en_US/champion/"
IMG_SPELL_URL = f"https://ddragon.leagueoflegends.com/cdn/{VERSION}/img/spell/"
IMG_PASSIVE_URL = f"https://ddragon.leagueoflegends.com/cdn/{VERSION}/img/passive/"

def fetch_details_clean(internal_id_list):
    full_data = []
    print(f"Traitement de {len(internal_id_list)} champions...")

    for champ_id in internal_id_list:
        url = f"{BASE_URL}{champ_id}.json"
        
        try:
            r = requests.get(url)
            if r.status_code != 200:
                print(f" Erreur URL pour {champ_id}")
                continue
            
            
            json_data = r.json()['data'][champ_id]
            
            row = {
                "InternalName": champ_id, # Pour faire la jointure
                "Name": json_data['name'],
                "Title": json_data['title'],
                "Lore": json_data['lore'],
                
                # Images
                "Image_Full": f"https://ddragon.leagueoflegends.com/cdn/img/champion/splash/{champ_id}_0.jpg",
                "Image_Loading": f"https://ddragon.leagueoflegends.com/cdn/img/champion/loading/{champ_id}_0.jpg",
                "Image_Sprite": json_data['image']['sprite'],
            }

            # Sorts
            keys = ["Q", "W", "E", "R"]
            for i, spell in enumerate(json_data['spells']):
                if i < 4:
                    k = keys[i]
                    row[f"Spell_{k}_Name"] = spell['name']
                    row[f"Spell_{k}_Desc"] = spell['description']
                    row[f"Spell_{k}_Icon"] = f"{IMG_SPELL_URL}{spell['image']['full']}"

            # Passif
            passive = json_data.get('passive', {})
            row["Passive_Name"] = passive.get('name', '')
            row["Passive_Desc"] = passive.get('description', '')
            if 'image' in passive:
                 row["Passive_Icon"] = f"{IMG_PASSIVE_URL}{passive['image']['full']}"

            full_data.append(row)

        except Exception as e:
            print(f"Erreur sur {champ_id}: {e}")

    return pl.DataFrame(full_data)

# UTILISATION
if 'liste_noms' in locals():
    df_details = fetch_details_clean(liste_noms)
    print("Terminé ! Résultat dans 'df_details'")
    print(df_details.head(2))

Patch : 16.2.1
Traitement de 172 champions...


KeyboardInterrupt: 

In [5]:
#sauvegarde du dataframe complet
df_details.write_csv("présentation_champions_blabla.csv", separator=";")

NameError: name 'df_details' is not defined

fusion potala des data frames

In [38]:
df_complet = df_final.join(df_details.drop([c for c in df_details.columns if c in df_final.columns and c != "InternalName"]), on="InternalName", how="left")
df_complet.write_csv("présentation_champions_complets.csv", separator=";")

In [45]:
 len(df_final)

172